In [1]:
import argparse
import glob
import os
from copy import copy
from pprint import pprint

import cv2
import numpy as np
import torch
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils import imwrite
from gfpgan import GFPGANer
from gfpgan.archs.stylegan2_clean_arch import ModulatedConv2d
from realesrgan import RealESRGANer
from tqdm import tqdm
from utils import *

from concrete.ml.torch.hybrid_model import HybridFHEModel

from utils import *

/home/celia/Desktop/Zama/concrete-internal/.dm/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


Overview of the GFP-GAN Architecture

The GFP-GAN pipeline is divided into 3 main components:

1. Face Cropping (restorer.face_helper): To detect and crop faces from input images.

    Composition:
    - 82 Standard Convolutional Layers with:
        + Kernel Sizes: (7, 7) or (3, 3) or (1, 1)
        + Strides: (2, 2) or (1, 1)
        + Padding: (3, 3) or (1, 1)


    - **Absence of Grouped or Dilated Convolutions or Depthwise Convolutions**

        + Grouped Convolutions: Convolutions where the input is divided into parts/groups, we have a set of filters for each group, the result is concatenated.
        (groups=1 by default).

        + Dilated Convolutions: Convolutions where the kernel is expanded by inserting zeros between its elements, increasing the receptive field without increasing the number of parameters.
        (dilation=1 by default).

    - **Modulated**:

        + The convolutional weights are dynamically adjusted (modulated) for each input sample based on a style vector.
        + This modulation allows the network to adapt its convolutional filters per sample, enabling more control over generated features.
        + Modulate Weights Process: For each ModulatedConv2d layer:
            - The style vector is transformed (usually via another linear layer) to obtain modulation weights.
            - These weights modulate the convolutional filters.
            - Demodulation: After modulation, weights can vary in magnitude, leading to instability during training. Demodulation normalizes the weights to maintain a consistent signal magnitude across the layers.


        ```python
        Style Vector (w)
                |
        Modulation Weights (s)
                |
        Modulated Weights (s * k)
                |
        (Optional) Demodulation
                |
        Convolution Operation
                |
        Output Feature Maps
        ```

2. Face Restoration (restorer.gfpgan): To restore and enhance the quality of cropped facial images.

    Composition:
    - 32 Linear Layers
    - 79 Standard Convolutional Layers with:
        + Kernel Sizes: (3, 3) or (1, 1)
        + Strides: (1, 1)
        + Padding: (1, 1)
    - 23 Modulated Convolutional Layers (ModulatedConv2d), with: Kernel Sizes: 3 or 11

3. Background Enhancement (restorer.upsampler): To enhance the background details of the images after face restoration.

    Composition:
    - 351 Standard Convolutional Layers with fixed configurations:
        + Kernel Size: (3, 3)
        + Stride: (1, 1)
        + Padding: (1, 1)


In [2]:
class Args:

    def __init__(self):

        self.input = "GFPGAN/inputs/whole_imgs"
        self.output = "results"
        self.version = "1.4"
        self.upscale = 5
        self.bg_upsampler = "realesrgan"
        self.bg_tile = 400
        self.suffix = None
        self.only_center_face = False
        self.aligned = False
        self.ext = "auto"
        self.weight = 0.5


args = Args()

In [3]:
img_list = sorted(glob.glob(f"{args.input}/*"))

os.makedirs(args.output, exist_ok=True)

assert len(img_list) >= 1

In [4]:
use_background_improvement = True

if args.bg_upsampler == "realesrgan":
    if use_background_improvement:

        half = True if torch.cuda.is_available() else False

        model = RRDBNet(
            num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2
        )
        # No linear modules in this model
        bg_upsampler = RealESRGANer(
            scale=2,  # Do not change this value
            model_path="https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth",
            model=model,
            tile=args.bg_tile,
            tile_pad=10,
            pre_pad=0,
            half=half,
        )  # need to set False in CPU mode

In [5]:
if args.version == "1":
    arch = "original"
    channel_multiplier = 1
    model_name = "GFPGANv1"
    url = "https://github.com/TencentARC/GFPGAN/releases/download/v0.1.0/GFPGANv1.pth"
elif args.version == "1.2":
    arch = "clean"
    channel_multiplier = 2
    model_name = "GFPGANCleanv1-NoCE-C2"
    url = "https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth"
elif args.version == "1.3":
    arch = "clean"
    channel_multiplier = 2
    model_name = "GFPGANv1.3"
    url = "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth"
    local_model_path = "GFPGANv1.3.pth"
elif args.version == "1.4":
    arch = "clean"
    channel_multiplier = 2
    model_name = "GFPGANv1.4"
    url = "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth"
    local_model_path = "GFPGANv1.4.pth"
elif args.version == "RestoreFormer":
    arch = "RestoreFormer"
    channel_multiplier = 2
    model_name = "RestoreFormer"
    url = "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/RestoreFormer.pth"
else:
    raise ValueError(f"Wrong model version {args.version}.")

# determine model paths
model_path = os.path.join("experiments/pretrained_models", model_name + ".pth")
if not os.path.isfile(model_path):
    model_path = os.path.join("gfpgan/weights", model_name + ".pth")
if not os.path.isfile(model_path):
    # download pre-trained models from url
    model_path = url

print(f"\n\n-----------------------: {model_path=}- {args.version=}\n")

restorer = GFPGANer(
    model_path=model_path,
    upscale=args.upscale,
    arch=arch,
    channel_multiplier=channel_multiplier,
    bg_upsampler=bg_upsampler,
)

print(f"-----------------------: {model_path=}- {args.version=}\n")

print(f"-----------------------: {restorer=}\n")

assert isinstance(restorer.gfpgan, torch.nn.Module) is True
assert isinstance(model, torch.nn.Module) is True



-----------------------: model_path='https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'- args.version='1.4'

-----------------------: model_path='https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'- args.version='1.4'

-----------------------: restorer=<gfpgan.utils.GFPGANer object at 0x71a565f1d520>



### Model 1

In [6]:
# Model 1: Face cropping and extraction
# using a FaceRestoreHelper with retinaface_resnet50
# No linear layers

face_helper_model = restorer.face_helper.face_det
face_helper_state_dict = restorer.face_helper.face_det.state_dict()

restorer.gfpgan =restorer.gfpgan

### Model 2

In [7]:
restorer_gfpgan_linear = extract_specific_module(
    restorer.gfpgan, dtype_layer=torch.nn.Linear, verbose=False
)

restorer_gfpgan_conv2d = extract_specific_module(
    restorer.gfpgan, dtype_layer=torch.nn.Conv2d, verbose=False
)

restorer_gfpgan_modulated_conv2d = extract_specific_module(
    restorer.gfpgan, dtype_layer=ModulatedConv2d, verbose=False
)


print(
    len(restorer_gfpgan_linear), len(restorer_gfpgan_conv2d), len(restorer_gfpgan_modulated_conv2d)
)

model = copy(restorer.gfpgan)

input_size = (3, 512, 512)

compile_size = 2
inputs = torch.randn((compile_size, *input_size))

hybrid_model = HybridFHEModel(
    model,
    [
        name
        for name, _ in restorer_gfpgan_linear
        if restorer.gfpgan.input_is_latent and "style_mlp" not in name
    ],
    verbose=2,
)
# Compile hybrid model
hybrid_model.compile_model(
    inputs,
    n_bits=8,
)

# summary(restorer.gfpgan, input_size=input_size)

32 79 23


In [8]:
output = hybrid_model(inputs)

In [9]:
input_size = (3, 256, 256)
inputs = torch.randn((compile_size, *input_size))
_ = hybrid_model(torch.randn((1, 3, 512, 512)))

### Model 3

In [10]:
isinstance(restorer.bg_upsampler.model, torch.nn.Module)

restorer_upsampler_linear = extract_specific_module(
    restorer.bg_upsampler.model, dtype_layer=torch.nn.Linear, verbose=False
)

restorer_upsampler_conv2d = extract_specific_module(
    restorer.bg_upsampler.model, dtype_layer=torch.nn.Conv2d, verbose=False
)

print(len(restorer_upsampler_linear), len(restorer_upsampler_conv2d))

0 351


In [11]:
for img_path in tqdm(img_list):
    # read image
    img_name = os.path.basename(img_path)
    print(f"Processing {img_name} ...")
    basename, ext = os.path.splitext(img_name)
    input_img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    print(f"{input_img.shape=}")

    # restore faces and background if necessary
    cropped_faces, restored_faces, restored_img = restorer.enhance(
        input_img,
        has_aligned=args.aligned,
        only_center_face=args.only_center_face,
        paste_back=True,
        weight=args.weight,
    )

    # print(f"{len(cropped_faces)=} | {len(restored_faces)=} | {restored_img.shape=}")

    # save faces
    for idx, (cropped_face, restored_face) in tqdm(enumerate(zip(cropped_faces, restored_faces))):
        # save cropped face
        save_crop_path = os.path.join(args.output, "cropped_faces", f"{basename}_{idx:02d}.png")
        imwrite(cropped_face, save_crop_path)
        # save restored face
        if args.suffix is not None:
            save_face_name = f"{basename}_{idx:02d}_{args.suffix}.png"
        else:
            save_face_name = f"{basename}_{idx:02d}.png"
        save_restore_path = os.path.join(args.output, "restored_faces", save_face_name)
        imwrite(restored_face, save_restore_path)
        # save comparison image
        cmp_img = np.concatenate((cropped_face, restored_face), axis=1)
        imwrite(cmp_img, os.path.join(args.output, "cmp", f"{basename}_{idx:02d}.png"))

    # save restored img
    if restored_img is not None:
        if args.ext == "auto":
            extension = ext[1:]
        else:
            extension = args.ext

        if args.suffix is not None:
            save_restore_path = os.path.join(
                args.output, "restored_imgs", f"{basename}_{args.suffix}.{extension}"
            )
        else:
            save_restore_path = os.path.join(
                args.output, "restored_imgs", f"{basename}.{extension}"
            )
        imwrite(restored_img, save_restore_path)
        break

print(f"Results are in the [{args.output}] folder.")

  0%|          | 0/3 [00:00<?, ?it/s]

Processing 00.jpg ...
input_img.shape=(1920, 1297, 3)
(1920, 1297, 3) iciii
a la fin (1920, 1297, 3)
MAINTENAT:  (1920, 1297, 3)
detect faces (1920, 1297, 3)
Inside detect faces: image.shape=torch.Size([1, 3, 1920, 1297])
self.half_inference=False, image.shape=torch.Size([1, 3, 1920, 1297])
type(bboxes)=<class 'numpy.ndarray'>, bboxes.shape=(2, 15)
2 khksdgkjhdbgkjhsbq
laaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa torch.Size([1, 3, 512, 512])
laaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa torch.Size([1, 3, 512, 512])
	Tile 1/20
	Tile 2/20
	Tile 3/20
	Tile 4/20
	Tile 5/20
	Tile 6/20
	Tile 7/20
	Tile 8/20
	Tile 9/20
	Tile 10/20
